In [1]:
import csv
import itertools
import pandas as pd
import numpy as np
import networkx as nx

In [2]:
papers=[]
with open("train.txt", "r") as f:
    for x in f:
        token_list=x.rstrip("\n").split()
        papers.append(token_list)

graph=[]

for paper in papers:
    for pair in itertools.combinations(paper, 2):
        graph.append(pair)

graph_df=pd.DataFrame(graph, columns=['Author_A','Author_B'])
#Calculate weights
g_df = graph_df.groupby(['Author_A','Author_B']).size().reset_index().rename(columns={0:'count'})
#create undirectional graph 
ng = nx.convert_matrix.from_pandas_edgelist(g_df, source= 'Author_A', target = 'Author_B', edge_attr= 'count')
print(nx.info(ng))


Name: 
Type: Graph
Number of nodes: 3767
Number of edges: 16036
Average degree:   8.5139


In [3]:
cnc = nx.common_neighbor_centrality(ng)

In [5]:
[print (a,b,c) for a,b,c in cnc]

3386 1486 150.67999999999995


KeyError: '516'

In [ ]:
# this step takes ages Link prediction metrics are very slow to calculate
feature_1 = []
for s,t in ng.edges():
    #not all nodes have proper metrics, it may throws erros, need to check the formulas
    try:
        metric = nx.common_neighbor_centrality(ng, [(s,t)])
        feature_1.append(metric[2])
    except:
        feature_1.append(0)



In [27]:
#getting all set of edges
r = csv.reader(open('no_headers_graph_pos.csv','r'))
edges = dict()
for edge in r:
    edges[(edge[0], edge[1])] = 1

In [19]:
%%time
###generating negative edges from given graph
import random
missing_edges = set([])
while (len(missing_edges)<16036): # such that same number as positive edges
    a=random.randint(1, 1862220)
    b=random.randint(1, 1862220)
    tmp = edges.get((a,b),-1)
    if tmp == -1 and a!=b:
        try:
            if nx.shortest_path_length(g,source=a,target=b) > 2: 
                
                missing_edges.add((a,b))
            else:
                continue  
        except:  
                missing_edges.add((a,b))              
    else:
        continue

Wall time: 59.8 ms


In [20]:
len(missing_edges)

16036